# re:Invent Chalk Talk - Building, Training, and Deploying Fast.ai Models Using Amazon SageMaker

## Setup

*This notebook was created and tested on an ml.p3.2xlarge notebook instance.*

Let's start by creating a SageMaker session and specifying:

* The **S3 bucket** and prefix that you want to use for training and model data. This should be within the same region as the Notebook Instance, training, and hosting.
* The **IAM role** arn used to give training and hosting access to your data. See the documentation for how to create these. Note, if more than one role is required for notebook instances, training, and/or hosting, please replace the sagemaker.get_execution_role() with appropriate full IAM role arn string(s). 

**IMPORTANT** please make sure the IAM role associated to your SageMaker notebook instance has the following managed IAM policies attached:

* **arn:aws:iam::aws:policy/AmazonSageMakerFullAccess**
* **arn:aws:iam::aws:policy/AmazonEC2ContainerRegistryFullAccess**

We also need to ensure there are no AWS credentials setup on the instance. We will set these up later in order to be able to train and deploy locally.

In [ ]:
%matplotlib inline
import os
import random
from io import BytesIO
import subprocess
from glob import glob

from IPython.display import display, HTML, Image

import matplotlib.pyplot as plt

from PIL import Image
import requests
import boto3

import sagemaker
from sagemaker.analytics import TrainingJobAnalytics
from sagemaker.local import local_session
from sagemaker.pytorch import PyTorch
from sagemaker.predictor import RealTimePredictor, json_deserializer
from sagemaker.pytorch import PyTorchModel
from sagemaker.utils import name_from_image

In [ ]:
def display_images(images, header=None, width="100%"):
    if type(width)==type(1): width = "{}px".format(width)
    html = ["<table style='width:{}'><tr>".format(width)]
    if header is not None:
        html += ["<th>{}</th>".format(h) for h in header] + ["</tr><tr>"]

    for image in images:
        html.append("<td><img src='{}' /></td>".format(image))
    html.append("</tr></table>")
    display(HTML(''.join(html)))

In [ ]:
! if [ -e ~/.aws/credentials ]; then rm ~/.aws/credentials; fi

In [ ]:
sagemaker_session = sagemaker.Session()

bucket_name = sagemaker_session.default_bucket()
prefix = 'sagemaker/DEMO-shirts-classification'

role = sagemaker.get_execution_role()

We also need to pull the needed Docker images from DockerHub that are specific for running fastai based models on SageMaker. 

There is a project with the source code and Dockerfile that can be found at the GitHub project: https://github.com/aws-samples/amazon-sagemaker-container-with-fastai.

Once we download the Docker images we then need to upload them to ECR so that they can be used by SageMaker.

In [ ]:
%%bash

# get the Dockerfile from GitHub
wget https://raw.githubusercontent.com/aws-samples/amazon-sagemaker-container-with-fastai/master/Dockerfile

IMAGE="sagemaker-fastai"

# parameters
FASTAI_VERSION="1.0"
PY_VERSION="py36"

# Get the account number associated with the current IAM credentials
account=$(aws sts get-caller-identity --query Account --output text)

if [ $? -ne 0 ]
then
    exit 255
fi

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-west-2}

# If the repository doesn't exist in ECR, create it.

aws ecr describe-repositories --repository-names "${IMAGE}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${IMAGE}" > /dev/null
fi

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ${region} --no-include-email)

# Get the login command from ECR in order to pull down the SageMaker PyTorch image
$(aws ecr get-login --registry-ids 520713654638 --region ${region} --no-include-email)

# loop for each architecture (cpu & gpu)
for arch in gpu cpu
do  
    echo "Building image with arch=${arch}, region=${region}"
    TAG="${FASTAI_VERSION}-${arch}-${PY_VERSION}"
    FULLNAME="${account}.dkr.ecr.${region}.amazonaws.com/${IMAGE}:${TAG}"
    docker build -t ${IMAGE}:${TAG} --build-arg ARCH="$arch"  --build-arg REGION="${region}"  .
    docker tag ${IMAGE}:${TAG} ${FULLNAME}
    docker push ${FULLNAME}
done

## Prepare Data

### Download training data to notebook instance

We will be utilizing a custom data set with a mixture of pictures of heavy metal t-shirts and sport shirts.

In [ ]:
%%bash
if [ ! -d data/shirts ]; then
    mkdir -p data/shirts
    wget -q https://s3-eu-west-1.amazonaws.com/sagemaker-934676248949-eu-west-1/data/shirts_imgs.tar.gz
    tar zxf shirts_imgs.tar.gz -C data/shirts
    rm shirts_imgs.tar.gz
fi

In [ ]:
DATA_PATH=f'{os.getcwd()}/data/shirts'

In [ ]:
%ls {DATA_PATH}

### View sample images

Let's look at some of the images in the folders.

In [ ]:
metal_img = random.choice(glob('data/shirts/metal/*.jpg'))
sport_img = random.choice(glob('data/shirts/sport/*.jpg'))

display_images([metal_img, sport_img],
       header=['Metal', 'Sport'], width="60%")

### Upload training data to S3

We are going to use the sagemaker.Session.upload_data function to upload our datasets to an S3 location. The return value inputs identifies the location -- we will use later when we start the training job.

In [ ]:
s3 = boto3.client('s3')
key = f'{prefix}/metal/'
response = s3.list_objects_v2(
        Bucket=bucket_name,
        Prefix=key,
)

if response['KeyCount'] > 0:
    print("Images exist in S3!")
    data_location=f's3://{bucket_name}/{prefix}'
else:
    print("Training images not uploaded to S3. Uploading now")
    data_location = sagemaker_session.upload_data(path=DATA_PATH, bucket=bucket_name, key_prefix=prefix)

print(f'training images location: {data_location}')

## Train on SageMaker

Now that we have tested the training and hosting locally, we are ready to train our model using the Amazon SageMaker training service.

Training a model on SageMaker with the Python SDK is done in a way that is similar to the way we trained it locally. This is done by changing our train_instance_type from `local` to one of our [supported EC2 instance types](https://aws.amazon.com/sagemaker/pricing/instance-types/).

In addition, we must now specify the ECR image URL, which we just pushed above.

Finally, our local training dataset has to be in Amazon S3 and the S3 URL to our dataset is passed into the `fit()` call.

Let's first fetch our ECR image url that corresponds to the image we just built and pushed.

In [ ]:
region = boto3.session.Session().region_name

client = boto3.client('sts')
account = client.get_caller_identity()['Account']

base_image_name = 'sagemaker-fastai'

In [ ]:
image_name = f'{account}.dkr.ecr.{region}.amazonaws.com/{base_image_name}:1.0-gpu-py36'
print(f'Using ECR image for SageMaker training: {image_name}')

Now we will create a `PyTorch` estimator object using the SageMaker SDK. The input parameters are almost exactly the same as when we trained locally except we will provide an instance type that will be a specfic instance type used to train the model using the SageMaker training service. In this specific example we will use the `ml.p3.2xlarge` instance type.

In [ ]:
estimator = PyTorch(entry_point='src/shirts-neo/train.py',
                    role=role,
                    train_instance_count=1,
                    train_instance_type='ml.p3.2xlarge',
                    image_name=image_name,
                    framework_version='1',
                    hyperparameters={
                        'epochs': 6, 
                        'batch-size': 64
                    },
                    metric_definitions=[
                        {'Name': 'valid:loss',     'Regex': '#quality_metric: host=\S+, epoch=\S+, valid_loss=(\S+)'},
                        {'Name': 'train:loss',     'Regex': '#quality_metric: host=\S+, epoch=\S+, train_loss=(\S+)'},
                        {'Name': 'valid:accuracy', 'Regex': '#quality_metric: host=\S+, epoch=\S+, accuracy=(\S+)'}
                    ])

In [ ]:
data_location=f's3://{bucket_name}/{prefix}'
print(f'Training data location: {data_location}')

In [ ]:
training_job_name=name_from_image('fastai-shirts')
estimator.fit(data_location, job_name=training_job_name)

Now we can plot the accuracy metric on a graph pulling the data from CloudWatch

### Graph training metrics from SageMaker

In [ ]:
# get the dataframe of training metrics
df = TrainingJobAnalytics(training_job_name=training_job_name,metric_names=['train:loss', 'valid:loss', 'valid:accuracy']).dataframe()

# plot the dataframe with matplotlib
fig, (ax1, ax2) = plt.subplots(1,2, sharey=False)
ax1.set_title('Loss')
ax1.set_ylabel('loss')
for key, grp in df.loc[df['metric_name'] != 'valid:accuracy'].groupby(['metric_name']):
    ax = grp.plot(ax=ax1, kind='line', x='timestamp', y='value', label=key)

ax2.set_title('Accuracy')
ax2.set_ylabel('accuracy')
df.loc[df['metric_name'] == 'valid:accuracy'].plot(ax=ax2, kind='line', x='timestamp', y='value', label='accuracy')
plt.tight_layout()

## Compile Model with SageMaker Neo

Now we will compile the model with SageMaker Neo.

In [ ]:
output_path = '/'.join(estimator.output_path.split('/')[:-1])
optimized_ic = estimator.compile_model(target_instance_family='ml_c5', 
                                input_shape={'actual_input_1':[1, 3, 224, 224]},  # Batch size 1, 3 channels, 224x224 Images.
                                output_path=output_path,
                                framework='onnx', 
                                framework_version='1.3')

In [ ]:
optimized_ic.name = 'deployed-shirts-classification'

## Host model with SageMaker

In [ ]:
optimized_ic_classifier = optimized_ic.deploy(initial_instance_count = 1,
                                              instance_type = 'ml.c5.4xlarge')

### Call SageMaker endpoint

Now we are ready to call the SageMaker endpoint to see if it is making correct inferences against some test data.

In [ ]:
# use on existing endpoint
#endpoint_name = 'fastai-shirts-2018-11-27-20-21-19-215'
#predictor = ImagePredictor(endpoint_name=endpoint_name, sagemaker_session=sagemaker_session)

In [ ]:
# Motorhead T-Shirt
#url = 'https://images.backstreetmerch.com/images/products/bands/clothing/mthd/bsi_mthd281.jpg'
# Judas Priest T-Shirt
#url = 'https://thumbs2.ebaystatic.com/d/l225/m/m7Lc1qRuFN3oFIlQla5V0IA.jpg'
# Iron Maiden T-Shirt
#url = 'https://www.ironmaidencollector.com/assets/pages/ab9ed-20180815_121042.jpg'
# All Blacks rugby jersey
#url = 'https://images.sportsdirect.com/images/products/38153703_l.jpg'
# Australia Rugby T-Shirt
#url = 'https://www.lovell-rugby.co.uk/products/products_580x387/40378.jpg'
# Chicago Bulls top
url = 'https://i.ebayimg.com/images/g/qc0AAOSwBahVN~qm/s-l300.jpg'
# Masters Golf Shirt
#url = 'https://s-media-cache-ak0.pinimg.com/originals/29/6a/15/296a15200e7dd3ed08e12d9052ea4f97.jpg'
img_bytes = requests.get(url).content
img = Image.open(BytesIO(img_bytes))
img

In [ ]:
response = optimized_ic_classifier.predict(img_bytes)
response

### Cleanup endpoint

When you're done with the endpoint, you should clean it up.

All of the training jobs, models and endpoints we created can be viewed through the SageMaker console of your AWS account.

In [ ]:
optimized_ic_classifier.delete_endpoint()

## Reference

- [How Amazon SageMaker interacts with your Docker container for training](https://docs.aws.amazon.com/sagemaker/latest/dg/your-algorithms-training-algo.html)
- [How Amazon SageMaker interacts with your Docker container for inference](https://docs.aws.amazon.com/sagemaker/latest/dg/your-algorithms-inference-code.html)
- [SageMaker Python SDK](https://github.com/aws/sagemaker-python-sdk)
- [Dockerfile](https://docs.docker.com/engine/reference/builder/)
- [PyTorch extending container example](https://github.com/awslabs/amazon-sagemaker-examples/blob/master/advanced_functionality/pytorch_extending_our_containers/pytorch_extending_our_containers.ipynb)
- [scikit-bring-your-own example](https://github.com/awslabs/amazon-sagemaker-examples/blob/master/advanced_functionality/scikit_bring_your_own/scikit_bring_your_own.ipynb)
- [SageMaker fast.ai container](https://github.com/aws-samples/amazon-sagemaker-container-with-fastai)
- [SageMaker fast.ai example](https://github.com/mattmcclean/sagemaker-fastai-example)
- [SageMaker PyTorch container](https://github.com/aws/sagemaker-pytorch-container)